# Step 01: Initialize Environment

This notebook validates the environment setup and prepares the workspace for the analysis cycle.

**Tasks:**
- Validate database connectivity
- Display cycle configuration
- Verify working directories
- Initialize step execution tracking

In [ ]:
# Import required modules
import sys
import os
from pathlib import Path

# Determine the notebook's actual directory
# In JupyterLab, __file__ is not available, but we can use the notebook server's root
# Get the path from the environment or construct it
if 'NOTEBOOK_DIR' in os.environ:
    # Custom environment variable (can be set in docker-compose)
    notebook_dir = Path(os.environ['NOTEBOOK_DIR'])
else:
    # Fallback: Assume standard structure and look for Active_ directories
    # Start from home and navigate to the workflows directory
    home = Path.home()
    workspace = home / 'workspace'
    
    # Check if we're in a docker environment
    if workspace.exists():
        # Look for Active_ cycle directories
        workflows = workspace / 'workflows'
        active_dirs = list(workflows.glob('Active_*'))
        
        if active_dirs:
            # Use the first Active_ directory found
            # This assumes there's only one active cycle
            notebook_dir = active_dirs[0] / 'notebooks' / 'Stage_01_Setup'
        else:
            # No active cycle found, use current directory
            notebook_dir = Path.cwd()
    else:
        # Not in Docker, use current working directory
        notebook_dir = Path.cwd()

print(f"Notebook directory: {notebook_dir}")

# Add workspace to path
workspace_path = notebook_dir.parent.parent.parent.parent
if str(workspace_path) not in sys.path:
    sys.path.insert(0, str(workspace_path))

from helpers import ux
from helpers.context import WorkContext
from helpers.cycle import get_cycle_by_name
from helpers.database import execute_query
from helpers.step import Step

## 1) Validate Database Connectivity

In [ ]:
# Test database connection
ux.header("Database Connectivity Check")

try:
    result = execute_query("SELECT version() as version")
    if not result.empty:
        db_version = result.iloc[0]['version']
        ux.success("✓ Database connection successful")
        ux.info(f"PostgreSQL Version: {db_version.split(',')[0]}")
    else:
        ux.error("✗ Database query returned no results")
        raise Exception("Database query returned empty result")
except Exception as e:
    ux.error(f"✗ Database connection failed: {str(e)}")
    raise

## 2) Validate Cycle Configuration

In [ ]:
# Initialize work context and display cycle information
ux.header("Cycle Configuration")

# Initialize context with explicit notebook directory
context = WorkContext(notebook_path=str(notebook_dir))

# Display context information
ux.subheader("Work Context")
context_info = [
    ["Cycle Name", context.cycle_name],
    ["Stage", context.stage_name],
    ["Step", context.step_name],
    ["Notebook Path", str(context.notebook_path)]
]
ux.table(context_info, headers=["Property", "Value"])

# Retrieve and display cycle details from database
try:
    cycle = get_cycle_by_name(context.cycle_name)
    
    ux.subheader("Cycle Details")
    cycle_info = [
        ["ID", cycle['id']],
        ["Name", cycle['cycle_name']],
        ["Status", cycle['status']],
        ["Created", cycle['created_ts'].strftime('%Y-%m-%d %H:%M:%S')]
    ]
    ux.table(cycle_info, headers=["Property", "Value"])
    
    if cycle['status'] != 'ACTIVE':
        ux.warning(f"⚠ Cycle status is '{cycle['status']}' (expected 'ACTIVE')")
    else:
        ux.success("✓ Cycle is active and ready")
        
except Exception as e:
    ux.error(f"✗ Failed to retrieve cycle information: {str(e)}")
    raise

## 3) Set Up Working Directories

In [ ]:
# Display working directory structure
ux.header("Working Directories")

# Get key directories from context
cycle_dir = context.cycle_directory
files_dir = cycle_dir / "files"
excel_config_dir = files_dir / "excel_configuration"
working_files_dir = files_dir / "working_files"
data_dir = files_dir / "data"
logs_dir = cycle_dir / "logs"

# Check and display directory status
directories = [
    ["Cycle Directory", cycle_dir, cycle_dir.exists()],
    ["Files Directory", files_dir, files_dir.exists()],
    ["Excel Configuration", excel_config_dir, excel_config_dir.exists()],
    ["Working Files", working_files_dir, working_files_dir.exists()],
    ["Data Directory", data_dir, data_dir.exists()],
    ["Logs Directory", logs_dir, logs_dir.exists()]
]

dir_info = []
all_exist = True
for name, path, exists in directories:
    status = "✓" if exists else "✗"
    dir_info.append([name, str(path), status])
    if not exists:
        all_exist = False

ux.table(dir_info, headers=["Directory", "Path", "Exists"])

if all_exist:
    ux.success("✓ All working directories are present")
else:
    ux.warning("⚠ Some directories are missing (they will be created as needed)")

## 4) Initialize Logging

In [ ]:
# Initialize step execution tracking
ux.header("Step Execution Tracking")

try:
    step = Step(context)
    
    # Check if step was already executed
    if step.executed:
        ux.warning("⚠ This step has already been executed")
        ux.info(f"Status: {step.status}")
        ux.info(f"Status Message: {step.status_message}")
        
        # Allow user to override and re-run if needed
        response = ux.yes_no("Do you want to re-run this step?", default="no")
        if response:
            ux.info("Re-running step...")
            step.start(force=True)
        else:
            ux.info("Step execution skipped")
            raise SystemExit("Step already completed")
    else:
        # Start the step
        step.start()
        ux.success(f"✓ Step '{context.step_name}' started successfully")
    
    # Log initialization complete
    step.log("Environment initialization completed successfully")
    step.log(f"Database connectivity: OK")
    step.log(f"Cycle validation: OK")
    step.log(f"Directory structure: OK")
    
    # Complete the step
    step.complete({
        'database_connected': True,
        'cycle_name': context.cycle_name,
        'cycle_status': cycle['status'],
        'directories_validated': all_exist
    })
    
    ux.success("\n" + "="*60)
    ux.success("✓ ENVIRONMENT INITIALIZATION COMPLETED SUCCESSFULLY")
    ux.success("="*60)
    ux.info("\nYou can now proceed to Step 02: Load Configuration")
    
except SystemExit:
    # User chose not to re-run
    pass
except Exception as e:
    ux.error(f"✗ Step initialization failed: {str(e)}")
    if 'step' in locals():
        step.fail(str(e))
    raise